<a href="https://colab.research.google.com/github/Harshita15887/Anndata/blob/main/notsoil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

print("⬆️ Please upload soil_dataset.zip")
soil_uploaded = files.upload()

print("⬆️ Please upload non_soil_dataset.zip")
non_soil_uploaded = files.upload()


⬆️ Please upload soil_dataset.zip


Saving archive (3).zip to archive (3).zip
⬆️ Please upload non_soil_dataset.zip


Saving archive (5).zip to archive (5).zip


In [2]:
import zipfile

# Extract soil dataset
with zipfile.ZipFile("archive (3).zip", 'r') as zip_ref:
    zip_ref.extractall("soil_dataset")

# Extract non-soil dataset
with zipfile.ZipFile("archive (5).zip", 'r') as zip_ref:
    zip_ref.extractall("non_soil_dataset")

print("✅ Both ZIP files extracted.")


✅ Both ZIP files extracted.


In [3]:
import os

# Create final dataset folders
base_path = "/content/final_dataset"
soil_types = ['Red', 'Clay', 'Black','Alluvial']
os.makedirs(base_path, exist_ok=True)

# Create folders for each soil type
for s in soil_types:
    os.makedirs(os.path.join(base_path, s), exist_ok=True)

# Create folder for not_soil
os.makedirs(os.path.join(base_path, "not_soil"), exist_ok=True)

print("✅ Final dataset folder structure created.")


✅ Final dataset folder structure created.


In [4]:
import shutil

soil_root = "/content/soil_dataset"

# Move soil images into their respective folders
for s in soil_types:
    source_dir = os.path.join(soil_root, s)
    target_dir = os.path.join(base_path, s)

    if os.path.exists(source_dir):
        for file in os.listdir(source_dir):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                shutil.move(os.path.join(source_dir, file), target_dir)

print("✅ Soil images moved.")


✅ Soil images moved.


In [5]:
for s in soil_types:
    source_dir = os.path.join(soil_root, s)
    target_dir = os.path.join(base_path, s)


In [6]:
non_soil_root = "/content/non_soil_dataset"
not_soil_target = os.path.join(base_path, "not_soil")

# Walk through all non-soil folders and move images
for root, dirs, files in os.walk(non_soil_root):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            shutil.move(os.path.join(root, file), not_soil_target)

print("✅ Non-soil images moved into 'not_soil'.")


✅ Non-soil images moved into 'not_soil'.


In [7]:
import zipfile

zip_path = "/content/archive (3).zip"
extract_path = "/content/soil_dataset"

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ ZIP extracted to:", extract_path)


✅ ZIP extracted to: /content/soil_dataset


In [8]:
import os

# See all folders and files in the extracted ZIP
for root, dirs, files in os.walk(extract_path):
    if any(f.lower().endswith(('.jpg', '.jpeg', '.png')) for f in files):
        print(f"{root} — {len(files)} image(s)")


/content/soil_dataset/Dataset/Train/Clay soil — 199 image(s)
/content/soil_dataset/Dataset/Train/Red soil — 264 image(s)
/content/soil_dataset/Dataset/Train/Alluvial soil — 528 image(s)
/content/soil_dataset/Dataset/Train/Black Soil — 231 image(s)
/content/soil_dataset/Dataset/test/Clay soil — 65 image(s)
/content/soil_dataset/Dataset/test/Red soil — 106 image(s)
/content/soil_dataset/Dataset/test/Alluvial soil — 54 image(s)
/content/soil_dataset/Dataset/test/Black Soil — 116 image(s)


In [9]:
import shutil

# Create unified target structure
base_path = "/content/final_dataset"
soil_types = ['clay', 'black', 'red', 'alluvial']

# Create destination folders
for s in soil_types:
    os.makedirs(os.path.join(base_path, s), exist_ok=True)

# Move images to respective folders based on name match
for soil_type in soil_types:
    found = False
    for root, dirs, files in os.walk(extract_path):
        if soil_type in root.lower():  # Match folder name like 'Black' or 'black_soil'
            found = True
            print(f"✅ Found {soil_type} folder at: {root}")
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    src = os.path.join(root, file)
                    dst = os.path.join(base_path, soil_type)
                    shutil.move(src, dst)
            break
    if not found:
        print(f"⚠️ {soil_type} folder not found!")


✅ Found clay folder at: /content/soil_dataset/Dataset/Train/Clay soil
✅ Found black folder at: /content/soil_dataset/Dataset/Train/Black Soil
✅ Found red folder at: /content/soil_dataset/Dataset/Train/Red soil
✅ Found alluvial folder at: /content/soil_dataset/Dataset/Train/Alluvial soil


In [10]:
for s in soil_types:
    path = os.path.join(base_path, s)
    print(f"{s}: {len(os.listdir(path))} images")


clay: 199 images
black: 228 images
red: 262 images
alluvial: 525 images


In [11]:
soil_types = ['clay', 'black', 'red', 'alluvial', 'not_soil']
base_path = "/content/final_dataset"

for folder in soil_types:
    folder_path = os.path.join(base_path, folder)
    num_files = len([
        f for f in os.listdir(folder_path)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ])
    print(f"{folder}: {num_files} images")


clay: 199 images
black: 228 images
red: 262 images
alluvial: 525 images
not_soil: 16130 images


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

base_path = "/content/final_dataset"
img_height, img_width = 150, 150  # resize images to this size
batch_size = 32


In [13]:
# Data augmentation & normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80-20 split
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 13877 images belonging to 9 classes.
Found 3467 images belonging to 9 classes.


In [14]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # output layer with softmax for multiclass
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,829,513 (18.42 MB)

 Trainable params: 4,829,513 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
epochs = 15

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)


Epoch 1/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 487s 1s/step - accuracy: 0.9868 - loss: 0.0468 - val_accuracy: 0.9792 - val_loss: 0.0709
Epoch 2/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 44s 100ms/step - accuracy: 1.0000 - loss: 0.0043 - val_accuracy: 0.9783 - val_loss: 0.0671
Epoch 3/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 561s 1s/step - accuracy: 0.9878 - loss: 0.0396 - val_accuracy: 0.9783 - val_loss: 0.0674
Epoch 4/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 43s 97ms/step - accuracy: 1.0000 - loss: 0.0360 - val_accuracy: 0.9835 - val_loss: 0.0499
Epoch 5/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 519s 1s/step - accuracy: 0.9888 - loss: 0.0376 - val_accuracy: 0.9786 - val_loss: 0.0666
Epoch 6/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 41s 94ms/step - accuracy: 1.0000 - loss: 0.0162 - val_accuracy: 0.9800 - val_loss: 0.0618
Epoch 7/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 564s 1s/step - accuracy: 0.9871 - loss: 0.0356 - val_accuracy: 0.9847 - val_loss: 0.0474
Epoch 8/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 43s 97ms/step - accuracy: 0.9688 - loss: 0.0458 - val

In [19]:

model.save("/content/soil_classifier_model.h5")
print("Model saved!")


Model saved!


In [21]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_soil_image(img_path, model, class_indices):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    x = image.img_to_array(img)
    x = x / 255.0
    x = np.expand_dims(x, axis=0)

    preds = model.predict(x)[0]
    class_idx = np.argmax(preds)
    confidence = preds[class_idx]

    # Get class label from index
    idx_to_class = {v: k for k, v in class_indices.items()}
    predicted_class = idx_to_class[class_idx]

    return predicted_class, confidence

# Example usage:
img_path = "/content/sun.jpeg"
predicted_class, confidence = predict_soil_image(img_path, model, train_generator.class_indices)
print(f"Predicted: {predicted_class} with confidence {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Predicted: red with confidence 1.00
